In [ ]:
!nvidia-smi

Tue Apr 11 02:32:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# 配置云盘
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# 修改工作路径
%cd drive/MyDrive/AIGC/chatrwkv

Mounted at /content/drive
/content/drive/MyDrive/AIGC/chatrwkv


In [ ]:
!mkdir raven
%cd raven
# 7B中文主体模型
!wget https://huggingface.co/BlinkDL/rwkv-4-raven/resolve/main/RWKV-4-Raven-7B-v7-ChnEng-20230404-ctx2048.pth
# 14B英文主体模型
!wget https://huggingface.co/BlinkDL/rwkv-4-raven/resolve/main/RWKV-4-Raven-14B-v8-EngAndMore-20230408-ctx4096.pth

In [ ]:
!pip install rwkv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.2 MB/s eta 0:00:00


In [ ]:
os.environ["RWKV_JIT_ON"] = '1'
os.environ["RWKV_CUDA_ON"] = '0' # if '1' then use CUDA kernel for seq mode (much faster)
from rwkv.model import RWKV                         # pip install rwkv
model = RWKV(model='/content/drive/MyDrive/AIGC/chatrwkv/raven/RWKV-4-Raven-7B-v7-ChnEng-20230404-ctx2048', strategy='cuda fp16i8')

out, state = model.forward([187, 510, 1563, 310, 247], None)   # use 20B_tokenizer.json
print(out.detach().cpu().numpy())                   # get logits

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 6

Loading /content/drive/MyDrive/AIGC/chatrwkv/raven/RWKV-4-Raven-7B-v7-ChnEng-20230404-ctx2048.pth ...
Strategy: (total 32+1=33 layers)
* cuda [float16, uint8], store 33 layers
0-cuda-float16-uint8 1-cuda-float16-uint8 2-cuda-float16-uint8 3-cuda-float16-uint8 4-cuda-float16-uint8 5-cuda-float16-uint8 6-cuda-float16-uint8 7-cuda-float16-uint8 8-cuda-float16-uint8 9-cuda-float16-uint8 10-cuda-float16-uint8 11-cuda-float16-uint8 12-cuda-float16-uint8 13-cuda-float16-uint8 14-cuda-float16-uint8 15-cuda-float16-uint8 16-cuda-float16-uint8 17-cuda-float16-uint8 18-cuda-float16-uint8 19-cuda-float16-uint8 20-cuda-float16-uint8 21-cuda-float16-uint8 22-cuda-float16-uint8 23-cuda-float16-uint8 24-cuda-float16-uint8 25-cuda-float16-uint8 26-cuda-float16-uint8 27-cuda-float16-uint8 28-cuda-float16-uint8 29-cuda-float16-uint8 30-cuda-float16-uint8 31-cuda-float16-uint8 32-cuda-float16-uint8 
emb.weight                        f16      cpu  50277  4096 


In [ ]:
########################################################################################################
# The RWKV Language Model - https://github.com/BlinkDL/RWKV-LM
########################################################################################################

import numpy as np
np.set_printoptions(precision=4, suppress=True, linewidth=200)
import types, torch
from torch.nn import functional as F
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_file("/content/drive/MyDrive/AIGC/chatrwkv/raven/20B_tokenizer.json")

args = types.SimpleNamespace()
args.MODEL_NAME = '/content/drive/MyDrive/AIGC/chatrwkv/raven/RWKV-4-Raven-7B-v7-ChnEng-20230404-ctx2048'
args.n_layer = 24
args.n_embd = 1024

context = "\nIn a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese."
NUM_TRIALS = 3
LENGTH_PER_TRIAL = 100
TEMPERATURE = 1.0
TOP_P = 0.85

########################################################################################################

class RWKV_RNN(torch.jit.ScriptModule):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.eval() # set torch to inference mode
        
        w = torch.load(args.MODEL_NAME + '.pth', map_location='cuda')
        for k in w.keys():
            if      '.time_' in k: w[k] = w[k].squeeze()
            if '.time_decay' in k: w[k] = -torch.exp(w[k].float()) # the real time decay is like e^{-e^x}
            else: w[k] = w[k].float() # convert to f32 type
        
        self.w = types.SimpleNamespace() # set self.w from w
        self.w.blocks = {}
        for k in w.keys(): # example: "blocks.0.att.time_first" => self.w.blocks[0].att.time_first
            parts = k.split('.')
            last = parts.pop()
            here = self.w
            for p in parts:
                if p.isdigit():
                    p = int(p)
                    if p not in here: here[p] = types.SimpleNamespace()
                    here = here[p]
                else:
                    if not hasattr(here, p): setattr(here, p, types.SimpleNamespace())
                    here = getattr(here, p)
            setattr(here, last, w[k])

    def layer_norm(self, x, w):
        return F.layer_norm(x, (self.args.n_embd,), weight=w.weight, bias=w.bias)

    @torch.jit.script_method
    def channel_mixing(self, x, state, i:int, time_mix_k, time_mix_r, kw, vw, rw):
        xk = x * time_mix_k + state[5*i+0] * (1 - time_mix_k)
        xr = x * time_mix_r + state[5*i+0] * (1 - time_mix_r)
        state[5*i+0] = x
        r = torch.sigmoid(rw @ xr)
        k = torch.square(torch.relu(kw @ xk)) # square relu, primer paper
        return r * (vw @ k)

    @torch.jit.script_method
    def time_mixing(self, x, state, i:int, time_mix_k, time_mix_v, time_mix_r, time_first, time_decay, kw, vw, rw, ow):
        xk = x * time_mix_k + state[5*i+1] * (1 - time_mix_k)
        xv = x * time_mix_v + state[5*i+1] * (1 - time_mix_v)
        xr = x * time_mix_r + state[5*i+1] * (1 - time_mix_r)
        state[5*i+1] = x
        r = torch.sigmoid(rw @ xr)
        k = kw @ xk
        v = vw @ xv
        
        aa = state[5*i+2]
        bb = state[5*i+3]
        pp = state[5*i+4]
        ww = time_first + k
        qq = torch.maximum(pp, ww)
        e1 = torch.exp(pp - qq)
        e2 = torch.exp(ww - qq)
        a = e1 * aa + e2 * v
        b = e1 * bb + e2
        wkv = a / b
        ww = pp + time_decay
        qq = torch.maximum(ww, k)
        e1 = torch.exp(ww - qq)
        e2 = torch.exp(k - qq)
        state[5*i+2] = e1 * aa + e2 * v
        state[5*i+3] = e1 * bb + e2
        state[5*i+4] = qq
        return ow @ (r * wkv)

    def forward(self, token, state):
        with torch.no_grad():
            if state == None:
                state = torch.zeros(self.args.n_layer * 5, self.args.n_embd)
                for i in range(self.args.n_layer): state[5*i+4] = -1e30 # -infinity
            
            x = self.w.emb.weight[token]
            x = self.layer_norm(x, self.w.blocks[0].ln0)
            for i in range(self.args.n_layer):
                att = self.w.blocks[i].att
                x = x + self.time_mixing(self.layer_norm(x, self.w.blocks[i].ln1), state, i, 
                    att.time_mix_k, att.time_mix_v, att.time_mix_r, att.time_first, att.time_decay, 
                    att.key.weight, att.value.weight, att.receptance.weight, att.output.weight)
                ffn = self.w.blocks[i].ffn
                x = x + self.channel_mixing(self.layer_norm(x, self.w.blocks[i].ln2), state, i, 
                    ffn.time_mix_k, ffn.time_mix_r, 
                    ffn.key.weight, ffn.value.weight, ffn.receptance.weight)
            
            x = self.w.head.weight @ self.layer_norm(x, self.w.ln_out)
            return x.float(), state

##########################################################################################################

def sample_logits(out, temperature=1.0, top_p=0.8):
    probs = F.softmax(out, dim=-1).numpy()
    sorted_probs = np.sort(probs)[::-1]
    cumulative_probs = np.cumsum(sorted_probs)
    cutoff = float(sorted_probs[np.argmax(cumulative_probs > top_p)])
    probs[probs < cutoff] = 0
    if temperature != 1.0:
        probs = probs.pow(1.0 / temperature)
    probs = probs / np.sum(probs)
    out = np.random.choice(a=len(probs), p=probs)
    return out

########################################################################################################

print(f'\nUsing CPU. Loading {args.MODEL_NAME} ...')
model = RWKV_RNN(args)

print(f'\nPreprocessing context (slow version. see v2/rwkv/model.py for fast version)')
init_state = None
for token in tokenizer.encode(context).ids:
    init_out, init_state = model.forward(token, init_state)

for TRIAL in range(NUM_TRIALS):
    print(f'\n\n--[ Trial {TRIAL} ]-----------------', context, end="")
    all_tokens = []
    out_last = 0
    out, state = init_out.clone(), init_state.clone()
    for i in range(LENGTH_PER_TRIAL):
        token = sample_logits(out, TEMPERATURE, TOP_P)
        all_tokens += [token]
        tmp = tokenizer.decode(all_tokens[out_last:])
        if '\ufffd' not in tmp: # only print when we have a valid utf-8 string
            print(tmp, end="", flush=True)
            out_last = i + 1
        out, state = model.forward(token, state)       
print('\n')


Using CPU. Loading /content/drive/MyDrive/AIGC/chatrwkv/raven/RWKV-4-Raven-7B-v7-ChnEng-20230404-ctx2048 ...


OutOfMemoryError: ignored

In [ ]:
# !git clone https://github.com/saharNooby/rwkv.cpp.git
%cd rwkv.cpp

!cmake -DBUILD_SHARED_LIBS=ON .
!cmake --build . --config Release

/content/drive/MyDrive/AIGC/chatrwkv/rwkv.cpp
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- x86 detected
-- Configuring done
-- Generating done
-- Build files have been written to: /content/drive/MyDrive/AIGC/chatrwkv/rwkv.cpp
[-33%] Built target ggml
[ 33%] Built target rwkv


In [ ]:
# 将pth格式转换为ggml
!python rwkv/convert_pytorch_to_ggml.py /content/drive/MyDrive/AIGC/chatrwkv/raven/RWKV-4-Raven-7B-v7-ChnEng-20230404-ctx2048.pth rwkv.cpp-Raven-7B.bin float16

Reading /content/drive/MyDrive/AIGC/chatrwkv/raven/RWKV-4-Raven-7B-v7-ChnEng-20230404-ctx2048.pth
Writing emb.weight, shape torch.Size([50277, 4096]), type torch.float16
Writing blocks.0.ln1.weight, shape torch.Size([4096]), type torch.float32
Writing blocks.0.ln1.bias, shape torch.Size([4096]), type torch.float32
Writing blocks.0.ln2.weight, shape torch.Size([4096]), type torch.float32
Writing blocks.0.ln2.bias, shape torch.Size([4096]), type torch.float32
Writing blocks.0.ln0.weight, shape torch.Size([4096]), type torch.float32
Writing blocks.0.ln0.bias, shape torch.Size([4096]), type torch.float32
Writing blocks.0.att.time_decay, shape torch.Size([4096]), type torch.float32
Writing blocks.0.att.time_first, shape torch.Size([4096]), type torch.float32
Writing blocks.0.att.time_mix_k, shape torch.Size([4096]), type torch.float32
Writing blocks.0.att.time_mix_v, shape torch.Size([4096]), type torch.float32
Writing blocks.0.att.time_mix_r, shape torch.Size([4096]), type torch.float32
Wr

In [ ]:
# 模型量化
!python rwkv/quantize.py rwkv.cpp-Raven-7B.bin rwkv.cpp-Raven-7B-Q4_1_O.bin 4

Loading model from 'rwkv.cpp-Raven-7B.bin'
                                      emb.weight - [ 4096, 50277], type =    F16 size =  392.789 MB
                             blocks.0.ln1.weight - [ 4096,     1], type =    F32 size =    0.016 MB
                               blocks.0.ln1.bias - [ 4096,     1], type =    F32 size =    0.016 MB
                             blocks.0.ln2.weight - [ 4096,     1], type =    F32 size =    0.016 MB
                               blocks.0.ln2.bias - [ 4096,     1], type =    F32 size =    0.016 MB
                             blocks.0.ln0.weight - [ 4096,     1], type =    F32 size =    0.016 MB
                               blocks.0.ln0.bias - [ 4096,     1], type =    F32 size =    0.016 MB
                         blocks.0.att.time_decay - [ 4096,     1], type =    F32 size =    0.016 MB
                         blocks.0.att.time_first - [ 4096,     1], type =    F32 size =    0.016 MB
                         blocks.0.att.time_mix_k - [ 4096

In [ ]:
# 对话应用
!python rwkv/chat_with_bot.py rwkv.cpp-Raven-7B-Q4_1_O.bin

Loading 20B tokenizer
System info: AVX = 1 | AVX2 = 1 | AVX512 = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 
Loading RWKV model
Processing 92 prompt tokens, may take a while
/content/drive/MyDrive/AIGC/chatrwkv/rwkv.cpp/rwkv/rwkv_cpp_model.py:100: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  state_out.storage().data_ptr(),
/content/drive/MyDrive/AIGC/chatrwkv/rwkv.cpp/rwkv/rwkv_cpp_model.py:101: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  log